In [3]:
import re
import math
from getpass import getpass

SPECIALS = r""" !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""

def assess_password(pw: str):
    length = len(pw)
    has_lower = bool(re.search(r"[a-z]", pw))
    has_upper = bool(re.search(r"[A-Z]", pw))
    has_digit = bool(re.search(r"\d", pw))
    has_special = bool(re.search(rf"[{re.escape(SPECIALS)}]", pw))

    # Length score: 0–4
    if length >= 16:
        length_score = 4
    elif length >= 12:
        length_score = 3
    elif length >= 8:
        length_score = 2
    elif length >= 6:
        length_score = 1
    else:
        length_score = 0

    # Variety score: up to 4 (one point per category met)
    variety_score = sum([has_lower, has_upper, has_digit, has_special])

    # Bonus for avoiding simple repeats or single-char lists
    penalties = 0
    if re.search(r"(.)\1{2,}", pw):  # 3+ same chars in a row
        penalties += 1
    if pw.isalpha() or pw.isdigit():
        penalties += 1

    raw_score = length_score + variety_score - penalties
    score = max(0, min(10, raw_score))  # clamp to 0–10

    if score <= 3:
        strength = "Weak"
    elif score <= 6:
        strength = "Fair"
    elif score <= 8:
        strength = "Strong"
    else:
        strength = "Very Strong"

    # Suggestions
    suggestions = []
    if length < 12:
        suggestions.append("Use at least 12 characters. 16+ is better.")
    if not has_lower:
        suggestions.append("Add lowercase letters.")
    if not has_upper:
        suggestions.append("Add uppercase letters.")
    if not has_digit:
        suggestions.append("Add numbers.")
    if not has_special:
        suggestions.append("Add special characters.")
    if re.search(r"(.)\1{2,}", pw):
        suggestions.append("Avoid repeating the same character three or more times.")
    if pw.isalpha() or pw.isdigit():
        suggestions.append("Mix letters, numbers, and symbols instead of just one type.")

    # Rough entropy estimate (based on character pools used)
    pool = 0
    if has_lower: pool += 26
    if has_upper: pool += 26
    if has_digit: pool += 10
    if has_special: pool += len(SPECIALS)
    entropy_bits = 0.0 if pool == 0 else length * math.log2(pool)

    return {
        "length": length,
        "has_lower": has_lower,
        "has_upper": has_upper,
        "has_digit": has_digit,
        "has_special": has_special,
        "score": score,
        "strength": strength,
        "entropy_bits": round(entropy_bits, 2),
        "suggestions": suggestions
    }

def print_report(result):
    print("\nPassword Report")
    print("----------------")
    print(f"Length: {result['length']}")
    print(f"Contains lowercase: {result['has_lower']}")
    print(f"Contains uppercase: {result['has_upper']}")
    print(f"Contains digits:    {result['has_digit']}")
    print(f"Contains special:   {result['has_special']}")
    print(f"Score (0–10):       {result['score']}")
    print(f"Strength:           {result['strength']}")
    print(f"Entropy (bits):     {result['entropy_bits']}")
    if result["suggestions"]:
        print("\nSuggestions:")
        for s in result["suggestions"]:
            print(f"• {s}")
    else:
        print("\nNice. No changes needed.")

if __name__ == "__main__":
    print("Password Complexity Checker")
    mode = input("Type (1) to paste your password, or (2) to enter hidden: ").strip()
    if mode == "1":
        pw = input("Enter password: ")
    else:
        pw = getpass("Enter password: ")

    report = assess_password(pw)
    print_report(report)


Password Complexity Checker
Type (1) to paste your password, or (2) to enter hidden: 1
Enter password: 12345

Password Report
----------------
Length: 5
Contains lowercase: False
Contains uppercase: False
Contains digits:    True
Contains special:   False
Score (0–10):       0
Strength:           Weak
Entropy (bits):     16.61

Suggestions:
• Use at least 12 characters. 16+ is better.
• Add lowercase letters.
• Add uppercase letters.
• Add special characters.
• Mix letters, numbers, and symbols instead of just one type.
